<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import userdata

# Define project information
PROJECT_ID = userdata.get('PROJECT_ID')
LOCATION = userdata.get('LOCATION')


with open('project_id', 'w') as p:
    p.write(PROJECT_ID)

In [ ]:
# import sys

# # Additional authentication is required for Google Colab
# if "google.colab" in sys.modules:
#     # Authenticate user to Google Cloud
#     from google.colab import auth

#     auth.authenticate_user()

In [ ]:
# можно попробовать на других платформах
# pip3 install google-cloud-aiplatform

!gcloud auth login
!gcloud config set project $(cat project_id)

In [145]:
import vertexai
from vertexai.generative_models import GenerativeModel


vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel("gemini-1.0-pro")

In [146]:
def load_about_company_text(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        content = f.read()
    return content


about_company_data = load_about_company_text('КТП_Test.txt')

In [191]:
def send_data(prompt):
    responses = model.generate_content(prompt, stream=True)
    data = ''
    for response in responses:
        data += response.text
    return data


about_company_prompt = about_company_data + 'это информация о том, что производит компания, напиши продукт, который производит компания, в одном предложении ответь на руском языке'
about_company = send_data(about_company_prompt)

print(about_company)

Компания производит комплектные трансформаторные подстанции различной мощности, с масляными и сухими трансформаторами, а также шкафы управления оперативным током, шкафы распределительные, устройства вводно-распределительные, системы быстродействующего автоматического ввода резерва.


In [192]:
import time
import pandas as pd
from IPython.display import display


df = pd.read_csv("Отчет по текущим тендерам.csv")

df = df[:6]

display(df)



def convert_to_dict(row):
  """
  Преобразует строку в словарь,
  используя значения столбцов в качестве ключей.
  """
  return {col: str(val).strip('"').strip("'") for col, val in zip(df.columns, row)}



data_store = df.apply(convert_to_dict, axis=1)



for d in data_store:
    name = d['Наименование лота']

    row_prompt =  f'{about_company} Относится ли {str(name)} к категории продукт для электростанции?'

    response = send_data(row_prompt)

    d['analyzed'] = response

    print(d)



,Дата внесения в таблицу,Объявление №,Лот №,Наименование лота,Способ закупок,Статус закупки,Дата начала,Дата окончания,Начальная максимальна цена,Кол-во,...,Руководитель,Менеджер,Решение,Комментарий,Допуск,Победитель,Цена,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,17.03.2023,2036727,NaN,КРУН 10 кВ Западная,NaN,Завершено,NaN,29.03.2023,"290 000 000,00",,...,Манатов А.Б.,Берик,участвуем,NaN,Допущен,"ТОО ""АЭТЗ""",NaN,NaN,NaN,NaN
1,17.03.2023,9499432-2,NaN,Устройства распределительные,NaN,Завершено,NaN,31.03.2023,"163 660 117,00",,...,Манатов А.Б.,Берик,участвуем,"Признана несостоявшейся в связи с тем, что к у...",Отклонен,Не состоялся,NaN,NaN,NaN,NaN
2,18.03.2023,2036981,NaN,КТП,NaN,Завершено,NaN,04.04.2023,"102 093 121,00",,...,Манатов А.Б.,Берик,участвуем,прошу отправить ОЛ либо проект для ознакомлени...,не участвовали,Протокол не доступен,NaN,NaN,NaN,NaN
3,20.03.2023,000010363 - eurasiantech,NaN,КТП,NaN,Завершено,NaN,03.04.2023,"30 322 563,00",,...,Манатов А.Б.,Берик,участвуем,прошу отправить ОЛ либо проект для ознакомлени...,Допущен,ТОО Bolashak Electric,NaN,NaN,NaN,NaN
4,20.03.2023,10460-eurasiantech,NaN,КСО-3,NaN,Завершено,NaN,05.04.2023,"2 198 600,00",,...,Манатов А.Б.,NaN,участвуем,"програли, не прошли по цене",Отклонен,ТОО ДИКОМ ЭНЕРДЖИ,NaN,NaN,NaN,NaN
5,20.03.2023,Р/00579/20/03/23,NaN,КТП,NaN,Завершено,NaN,31.03.2023,NaN,,...,Манатов А.Б.,NaN,не участвуем,Нет решения от ПТО и производства,не участвовали,Протокол не доступен,NaN,NaN,NaN,NaN


{'Дата внесения в таблицу': '17.03.2023', 'Объявление №': '2036727', 'Лот №': 'nan', 'Наименование лота': 'КРУН 10 кВ Западная', 'Способ закупок': 'nan', 'Статус закупки ': 'Завершено ', 'Дата начала': 'nan', 'Дата окончания': '29.03.2023', 'Начальная максимальна цена ': '290\xa0000\xa0000,00', 'Кол-во': '\xa0', 'Ед.изм.': '\xa0', 'Сумма': 'nan', 'Заказчик': 'ТОО Кокшетау-Энерго', 'Регион': 'Кокшетау', 'Источник': 'https://www.ets-tender.kz/market/view.html?id=2036727', 'Контакты заказчика': 'nan', 'Тендерный специалист ': 'Приходько', 'Руководитель ': 'Манатов А.Б.', 'Менеджер ': 'Берик', 'Решение': 'участвуем', 'Комментарий': 'nan', 'Допуск ': 'Допущен', 'Победитель': 'ТОО "АЭТЗ', 'Цена': 'nan', 'Unnamed: 24': 'nan', 'Unnamed: 25': 'nan', 'Unnamed: 26': 'nan', 'analyzed': '## Относится ли КРУН 10 кВ Западная к категории продукт для электростанции?\n\n## Ответ:\n\n**Да, КРУН 10 кВ Западная можно отнести к категории продукт для электростанции.**\n\n**КРУН 10 кВ Западная** - это комплек